Pairs Trading in Crytpo, Event Driven Backtesting and Evaluation

In [ ]:
import warnings
warnings.filterwarnings('ignore')

1. Data processing & EDA

In [ ]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import datetime
import os
import shutil
from decimal import Decimal
import fsspec
import pandas as pd

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/tencars/392-crypto-currency-pairs-at-minute-resolution")

Skipping, found downloaded files in "./392-crypto-currency-pairs-at-minute-resolution" (use force=True to force download)


In [ ]:
DATA_DIR = "./392-crypto-currency-pairs-at-minute-resolution"
fs = fsspec.filesystem('file')
raw_files = fs.glob(f"{DATA_DIR}/*.csv")
assert raw_files, f"Unable to find any histdata files in directory {DATA_DIR}"

I found that CSV file with "_usd.csv" is the relatively latest spot price data, so I will use spot price data for correlation calculation and long/short portfolio calculation

In [ ]:
spot = [link for link in raw_files if link.endswith('_usd.csv') ] # for correlation calculation

I see the time interval in these files concentrate from 2023-06-03 12:00:00 to 2023-06-11 12:00:00, so I will use interval 2023-06-03 12:00:00 => 2023-06-09 12:00:00 of the data for correlation calculation and interval 2023-06-09 12:00:00 => 2023-06-11 12:00:00 for backtesting

In [ ]:
#use multi threading to read all the csv files with file path array spot to pandas dataframe, with index as datetime, symbol as column name and close price as value, and then concatenate all the dataframes into one dataframe

import os
import pandas as pd
from tqdm import tqdm
from typing import List
import dask.dataframe as dd
import multiprocessing

def read_csv_to_df(file_path: str) -> pd.DataFrame:
  df = pd.read_csv(file_path, index_col = 'time')
  df = df[['close']]
  df.columns = [file_path.split('/')[-1].replace(".csv","")]
  df.index = pd.to_datetime(df.index, unit='ms')

  # assume df is a dataframe with a datetime index
  start_time = '2023-06-03 12:00:00'
  end_time = '2023-06-11 12:00:00'
  # create a new datetime index at 5-minute intervals
  new_index = pd.date_range(start=start_time, end=end_time, freq='5T')
  # resample the dataframe to the new index
  df_resampled = df.resample('5T').last().reindex(new_index)
  # fill missing values with the last observed value
  df_resampled.fillna(method='ffill', inplace=True)

  return df_resampled

def read_csvs_to_df(file_paths: List[str]) -> pd.DataFrame:
    with multiprocessing.Pool() as pool:
        dfs = list(tqdm(pool.map(read_csv_to_df, file_paths), total=len(file_paths)))
    return dfs

temp = read_csvs_to_df(spot)

spot_df = dd.concat(temp, axis=1).compute()


100%|██████████| 67/67 [00:00<00:00, 370736.63it/s]


In [ ]:
# count the number of missing values in each column
na_counts = spot_df.isna().sum()

# drop columns with more than 0 NaN value
spot_df = spot_df.drop(columns=na_counts[na_counts >= 1].index)

In [ ]:
start_time = '2023-06-03 12:00:00'
end_time = '2023-06-09 12:00:00'

# Convert the start_time and end_time strings to datetime objects
start_time = pd.to_datetime(start_time)
end_time = pd.to_datetime(end_time)

# Use boolean indexing to slice the DataFrame
spot_df_corr_test = spot_df[(spot_df.index >= start_time) & (spot_df.index <= end_time)]